In [1]:
import sys
import os

import cv2
from keras.models import load_model
import numpy as np

from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.inference import load_image
from utils.preprocessor import preprocess_input

from keras.preprocessing import image

import cv2
import math
import numpy as np

import matplotlib
%matplotlib inline
from matplotlib import pylab as plt
import matplotlib.patches as patches

from IPython.display import clear_output

import face_recognition
from scipy.interpolate import spline

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import time
from tqdm import tqdm_notebook as tqdm
from os.path import join as pj
import json

Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8194767159578497610
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 7958219981
locality {
  bus_id: 1
}
incarnation: 6564811094157515207
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:02:00.0"
]


In [2]:
image_path = sys.argv[1]
detection_model_path = '../trained_models/detection_models/haarcascade_frontalface_default.xml'
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
gender_model_path = '../trained_models/gender_models/simple_CNN.81-0.96.hdf5'
emotion_labels = get_labels('fer2013')
gender_labels = get_labels('imdb')
font = cv2.FONT_HERSHEY_SIMPLEX

# hyper-parameters for bounding boxes shape
gender_offsets = (30, 60)
gender_offsets = (10, 10)
emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

# loading models
face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
gender_classifier = load_model(gender_model_path, compile=False)

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]
gender_target_size = gender_classifier.input_shape[1:3]

In [3]:
def detect_face(image, video_file_path, frame_id):
    to_save = np.random.rand() > 0.99
    if not to_save:
        image = cv2.resize(image, (0,0), fx=0.25, fy=0.25)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')

    
    faces = detect_faces(face_detection, gray_image)
    if len(faces) == 0:
        return {}

    data = {}

    # emotion
    for face_coordinates in faces:
        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]

        gray_face = cv2.resize(gray_face, (emotion_target_size))

        gray_face = preprocess_input(gray_face, True)
        gray_face = np.expand_dims(gray_face, 0)
        gray_face = np.expand_dims(gray_face, -1)
        emotion_label_arg = np.argmax(emotion_classifier.predict(gray_face))
        emotion_text = emotion_labels[emotion_label_arg]

        data['emotion'] = emotion_text
        data['n_faces'] = len(faces)
        if not to_save:
            break  # only one face

        # save some images
        if to_save:
            draw_bounding_box(face_coordinates, image, (0, 0, 255))
            draw_text(face_coordinates, image, emotion_text, (0, 0, 255), 0, -10, 1, 2)
    
    if to_save:
        name = os.path.basename(video_file_path)[:-4]
        if not os.path.exists('./pics/{}'.format(name)):
            os.mkdir('./pics/{}'.format(name))
        plt.imsave('./pics/{}/{}.png'.format(name, frame_id), cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    return data

In [4]:
def get_result(video_file_path):
    cap = cv2.VideoCapture(video_file_path)
    frame_rate = cap.get(5)  # frame rate
    frame_counter = 0

    result = []
    while cap.isOpened():
        frame_id = cap.get(1)
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % math.floor(frame_rate) == 0:
            frame_counter += 1
            try:
                result.append(detect_face(frame, video_file_path, frame_id))
            except Exception as e:
                print('Failed on frame. Reason: {}', frame_id, e)
                result.append({})

#             if frame_counter > 1 * 60:
#                 break

    cap.release()
    
    return result

In [5]:
def get_filepath_by_idx():
    filepath_by_idx = {}

    ROOT = '/data/junction'
    for idx in range(1, 1000):
        filenames = []
        for root, dirs, files in os.walk('{}/{}'.format(ROOT, idx)):
            for file in files:
                filenames.append(pj(root, file))
        candidates = [filename for filename in filenames if os.path.splitext(filename)[1] in ['.avi', '.mkv']]
        if len(candidates) != 1:
#             print('! {} Shit candidates {} != 1'.format(idx, len(candidates)))
            for filename in filenames:
                print(filename)
        else:
            filepath_by_idx[idx] = candidates[0]
            
    return filepath_by_idx

In [6]:
filepath_by_idx = get_filepath_by_idx()

In [7]:
for i, video_file_path in tqdm(list(filepath_by_idx.items())[68:]):
    done_idxs = os.listdir('./results')
    name = '{}.json'.format(i)
    if name in done_idxs:
        print('{} is already done'.format(name))
        continue
    
    result = get_result(video_file_path)
    with open('./results/' + name, 'w') as fout:
        json.dump(result, fout)

A Jupyter Widget

71.json is already done
72.json is already done
73.json is already done
74.json is already done
75.json is already done
76.json is already done
77.json is already done
78.json is already done
79.json is already done
80.json is already done
81.json is already done
82.json is already done
83.json is already done
84.json is already done
85.json is already done
86.json is already done
87.json is already done
88.json is already done
89.json is already done
90.json is already done
91.json is already done
92.json is already done
93.json is already done
94.json is already done
95.json is already done
96.json is already done
97.json is already done
98.json is already done
100.json is already done
251.json is already done
252.json is already done
253.json is already done
254.json is already done



In [8]:
!ls -l results/

total 5388
-rwxrwxrwx 1 root root  58839 Nov 25 16:07 1.json
-rwxrwxrwx 1 root root  42233 Nov 25 16:44 10.json
-rw-r--r-- 1 root root  14554 Nov 26 04:27 100.json
-rwxrwxrwx 1 root root  94210 Nov 25 16:47 11.json
-rwxrwxrwx 1 root root  46907 Nov 25 16:49 12.json
-rwxrwxrwx 1 root root  35385 Nov 25 16:50 13.json
-rwxrwxrwx 1 root root  68706 Nov 25 16:53 14.json
-rwxrwxrwx 1 root root  95774 Nov 25 16:57 15.json
-rwxrwxrwx 1 root root  68404 Nov 25 16:59 16.json
-rwxrwxrwx 1 root root  54830 Nov 25 17:01 17.json
-rwxrwxrwx 1 root root  54583 Nov 25 17:02 18.json
-rwxrwxrwx 1 root root  66682 Nov 25 17:07 19.json
-rwxrwxrwx 1 root root  70107 Nov 25 16:11 2.json
-rwxrwxrwx 1 root root  37063 Nov 25 17:08 20.json
-rwxrwxrwx 1 root root  36611 Nov 25 17:10 21.json
-rwxrwxrwx 1 root root  42044 Nov 25 17:12 22.json
-rwxrwxrwx 1 root root  80454 Nov 25 17:14 23.json
-rwxrwxrwx 1 root root  51616 Nov 25 17:18 24.json
-rwxrwxrwx 1 root root  62630 Nov 25 17:20 25.json
-rw-r--r-- 1 root roo

In [9]:
# !rm -rf results/*